## Handling Select2 Controls in Selenium WebDriver
[Select2](https://github.com/select2/select2) is a jQuery based replacement for select boxes. This article will demonstrate how Selenium webdriver can handle Select2 by manipulating the first such selection box in the [Examples page](https://select2.github.io/examples.html) of Select2.


### Creating an instance of Selenium webdriver equipped with Firefox Extensions
[Firebug](http://getfirebug.com/) and [FirePath](https://addons.mozilla.org/ja/firefox/addon/firepath/) are very helpful Firefox extensions that I want to use in this demonstration, so I will make Selenium launch a Firefox browser equipped with these extensions.

In [2]:
import os
from marigoso import Test
request = {
    'firefox': {
        'extensions_path': os.path.join(os.getcwd(), 'tools', 'firefox'),
        'extensions':      ['firebug@software.joehewitt.com.xpi', 'FireXPath@pierre.tholence.com.xpi'],
    }
}
browser = Test(request).launch_browser('Firefox')

Note that in order for the extensions to be installed in the browser, you need to either specify an extension enabled Firefox profile to Selenium or you specify the location and name of Firefox extensions you want to install. In the above example, I have Firebug and FirePath files stored in 'tools\firefox' folder so I can just specify the location and filenames of the extensions.

### Navigate to Select2 Examples page

In [3]:
browser.get_url('https://select2.github.io/')
browser.press("Examples")

### Identify the locator for the Selection Box
Right click on the first Select2 box and select '**Inspect Element with Firebug**'

![blank](https://github.com/ldiary/marigoso/blob/master/notes/images/selec2_01.PNG?raw=true)

Firebug will then display and highlight the HTML source of the Selection Box as well as highlight the control itself if you hover your mouse to the HTML source.

![blank](https://github.com/ldiary/marigoso/blob/master/notes/images/selec2_02.PNG?raw=true)

We now have the task of figuring out what locator we can use to locate this Selection Box. The Selection Box is a 'span' element with an id="select2-jnw9-container", we can surely make use of this id attribute. However, it appears that this id is randomly generated so I made a slight modification to make sure my locator will still work even if the page is refreshed.

### Verify the adopted locator works
In the Firebug window, click on 'FirePath' tab. Click on the dropdown before the input box and select 'CSS:'. Then enter "[id^='select2']" in the input box and press Enter key.

![blank](https://github.com/ldiary/marigoso/blob/master/notes/images/selec2_03.PNG?raw=true)

Firebug will now display the same thing as before, but notice now that at the lower left part of Firebug window it says '17 matching nodes'. This means we have 17 such Selection Box that can be located using my chosen selector. However, this time we are only interested on the first Selection Box, so I think my chosen selector is still useful.

The ultimate way to verify that the locator works is to feed it to Selenium and run it. So we execute the following command.

In [4]:
browser.press("css=[id^='select2']" )

If the Selection Dropdown appears upon executing the above command, then we are on the right track. You can run the above command several times to confirm the closing and opening of the selection dropdown.

### Identify the locator for the Selection Dropdown
We now need to identify the locator for the Selection Dropdown. We do this by clicking back on the 'HTML' tab in the Firebug window and observing that when you manually click on the Selection Box another 'span' element is dynamically being added at the buttom of the HTML source.

![blank](https://github.com/ldiary/marigoso/blob/master/notes/images/selec2_04.PNG?raw=true)

We can use previous technique of locating the Selection Box above to arrive to a conclusion that the locator for Selection Dropdown could be 'css=span.select2-dropdown > span > ul'. Note that in this case we specifically located until the 'ul' tag element. This is because the options for Select2 are not 'option' tag elements, instead they are 'li' elements of a 'ul' tag.

![blank](https://github.com/ldiary/marigoso/blob/master/notes/images/selec2_05.PNG?raw=true)

### Verify that both Selection Box and Dropdown works
After all this hardwork of figuring out the best locators for Selection Box and Selection Dropdown, we then test it to see if we can now properly handle Select2. Marigoso offers two syntax for performing the same action.

#### select_text
We can use the usual select_text function by just appending the Select Dropdown locator at the end.

In [5]:
browser.select_text("css=*[id^='select2']", "Nevada", 'css=span.select2-dropdown > span > ul')

#### select2
We can also use the select2 function of Marigoso by swapping the order of the Selection Dropdown locator and the value of the text you want to select.

In [6]:
browser.select2("css=*[id^='select2']", 'css=span.select2-dropdown > span > ul', "Hawaii")

## Final Solution
Finally, here again is the summary of the necessary commands used in this demonstration.

In [7]:
import os
from marigoso import Test
request = {
    'firefox': {
        'extensions_path': os.path.join(os.getcwd(), 'tools', 'firefox'),
        'extensions':      ['firebug@software.joehewitt.com.xpi', 'FireXPath@pierre.tholence.com.xpi'],
    }
}
browser = Test(request).launch_browser('Firefox')
browser.get_url('https://select2.github.io/')
browser.press("Examples")
browser.select_text("css=*[id^='select2']", "Nevada", 'css=span.select2-dropdown > span > ul')
browser.quit()